In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Common Includes

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers, regularizers
import tensorflow_datasets as tfds
print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")
!lscpu |grep 'Model name'
!nvidia-smi

Importing the cifar10 dataset and creating an input pipeline

Downloading and preparing dataset 162.17 MiB (download: 162.17 MiB, generated: 132.40 MiB, total: 294.58 MiB)

In [9]:
AUTOTUNE = tf.data.AUTOTUNE
BATCH_SIZE = 64
BUFFER_SIZE = 1000
MAX_BRIGHT_DELTA = 0.1

(ds_train, ds_test), ds_info = tfds.load(
    name="cifar10",
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)
print(ds_info)
fig = tfds.show_examples(ds_train, ds_info, rows=4, cols=4)

def normalize_image(image, label):
    return tf.cast(x=image, dtype=tf.float32)/255.0, label
    return image, label
def rgb_to_gray(image, label):
    image = tf.image.rgb_to_grayscale(image)
    return image, label
def augment(image, label):
    image = tf.image.random_brightness(image, max_delta=MAX_BRIGHT_DELTA)
    #image = tf.image.random_contrast(image, lower=CONTRAST_LOWER, upper=CONTRAST_UPPER) #bad
    image = tf.image.random_flip_left_right(image) #50%
    return image, label

ds_train = ds_train.map(map_func=rgb_to_gray, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.map(map_func=normalize_image, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.map(map_func=augment, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.shuffle(buffer_size=BUFFER_SIZE)
ds_train = ds_train.batch(batch_size=BATCH_SIZE)
ds_train = ds_train.prefetch(buffer_size=AUTOTUNE)

ds_test = ds_test.map(map_func=rgb_to_gray, num_parallel_calls=AUTOTUNE)
ds_test = ds_test.map(map_func=normalize_image, num_parallel_calls=AUTOTUNE)
ds_test = ds_test.batch(batch_size=BATCH_SIZE)
ds_test = ds_test.prefetch(buffer_size=AUTOTUNE)

Creating the LeNet-5 model

In [8]:
INPUT_SHAPE = (32,32,1)
LEARNING_RATE = 0.001


model = keras.Sequential([
    layers.Input(shape=INPUT_SHAPE),
    layers.Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding="valid"),
    layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'),
    layers.Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid'),
    layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'),
    layers.Flatten(),
    layers.Dense(units=120, activation='tanh'),
    layers.Dense(units=84, activation='tanh'),
    layers.Dense(units=10)
])

EPOCHS = 20

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    metrics=['accuracy']
)

model.summary()

Calculating the FLOPs of the models

In [ ]:
from tensorflow.python.framework.convert_to_constants import  convert_variables_to_constants_v2_as_graph

def get_flops(model):
    concrete = tf.function(lambda inputs: model(inputs))
    concrete_func = concrete.get_concrete_function(
        [tf.TensorSpec([1, *inputs.shape[1:]]) for inputs in model.inputs])
    frozen_func, graph_def = convert_variables_to_constants_v2_as_graph(concrete_func)
    with tf.Graph().as_default() as graph:
        tf.graph_util.import_graph_def(graph_def, name='')
        run_meta = tf.compat.v1.RunMetadata()
        opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()
        flops = tf.compat.v1.profiler.profile(graph=graph, run_meta=run_meta, cmd="op", options=opts)
        return flops.total_float_ops
    
# The //2 is necessary since `profile` counts multiply and accumulate
# as two flops, here we report the total number of multiply accumulate ops    
print("The FLOPs is:{}".format(get_flops(model)//2) ,flush=True)

Training the model (15s per Epoch)

In [ ]:
model.fit(ds_train, epochs=EPOCHS, verbose=1)

Evaluating the Model

In [ ]:
model.evaluate(ds_test, verbose=1)

Evaluate the model with batch_size=1, to get the single batch average (1/100) inference time, on cpu

In [12]:
(ds_test_single), ds_info = tfds.load(
    name="cifar10",
    split="test[0:100]",
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

ds_test_single = ds_test_single.map(map_func=rgb_to_gray, num_parallel_calls=AUTOTUNE)
ds_test_single = ds_test_single.map(map_func=normalize_image, num_parallel_calls=AUTOTUNE)
ds_test_single = ds_test_single.batch(batch_size=1)
ds_test_single = ds_test_single.prefetch(buffer_size=AUTOTUNE)

with tf.device('/cpu:0'):
    model.evaluate(ds_test_single, verbose=1)

Save the model, zip it and remove the folder to save space

In [ ]:
Model_Name = "LeNet5_Cifar10_47pct_0_5MF"
os.environ["MODELNAME"] = Model_Name
os.environ["MODELNAMEZIP"] = Model_Name + ".zip"
model.save("./" + Model_Name)
!zip -r ./$MODELNAMEZIP ./$MODELNAME
!rm -rf ./$MODELNAME